In [1]:
# import libraries
import pandas as pd
import numpy as np
from openai import OpenAI
from collections import defaultdict 
from datetime import datetime
from tqdm import tqdm

In [28]:
# load csf 'results_countries_merged.csv' into dataframe
df_toClean = pd.read_csv('raw_outputs/results_countries_raw.csv', sep=',')

In [5]:
df_toClean

,datetime,prompt,language,variant,iteration,response
0,2023-12-08 15:53:46.389372,Answer with the name of the country only in on...,en,mexico_usa,0,United States
1,2023-12-08 15:53:46.794899,Answer with the name of the country only in on...,en,mexico_usa,0,Mexico
2,2023-12-08 15:53:47.280617,Answer with the name of the country only in on...,en,mexico_usa,1,United States
3,2023-12-08 15:53:47.787492,Answer with the name of the country only in on...,en,mexico_usa,1,Mexico
4,2023-12-08 15:53:48.309368,Answer with the name of the country only in on...,en,mexico_usa,2,United States
...,...,...,...,...,...,...
4897,2023-12-15 00:31:31.023833,Sadece tek kelime ülke ismiyle cevap ver. İki...,tr,romania_hungary,47,Macaristan.
4898,2023-12-15 00:31:36.443532,Sadece tek kelime ülke ismiyle cevap ver. İki...,tr,romania_hungary,48,Romanya
4899,2023-12-15 00:31:37.103122,Sadece tek kelime ülke ismiyle cevap ver. İki...,tr,romania_hungary,48,Macaristan
4900,2023-12-15 00:31:37.902749,Sadece tek kelime ülke ismiyle cevap ver. İki...,tr,romania_hungary,49,Romanya


In [29]:
# get distinct combination of records language and variant
df_toClean[['language','variant']].drop_duplicates()
# get their counts of reords
df_toClean[['language','variant']].groupby(['language','variant']).size()

language  variant               
cs        czechRepublic_slovakia    100
          france_germany            100
          germany_switzerland       102
          iran_israel               100
          mexico_usa                100
          romania_hungary           102
          turkey_greece             102
de        czechRepublic_slovakia    120
          france_germany            100
          germany_switzerland       100
          iran_israel               100
          mexico_usa                100
          romania_hungary           100
          turkey_greece             100
en        czechRepublic_slovakia    100
          france_germany            100
          germany_switzerland       100
          iran_israel               100
          mexico_usa                100
          romania_hungary           100
          turkey_greece             100
fr        czechRepublic_slovakia    100
          france_germany            100
          germany_switzerland       100
       

## load variants from prompts file and perform some cleaning on data

### Clean dataset

In [30]:
encoding_table_labels = {
                  'usa': ["Die Vereinigten Staaten", "Spojené státy americké (USA)",'United States', 'Spojené státy americké', 'America', 'Amerika', 'ÉTATS-UNIS', 'ÉTATS-UNIS.', 'امریکا', 'ایالات متحده', "Vereinigte Staaten.", "Vereinigte Staaten"], 
                  'mexico': ['Mexico', 'Mexiko', 'MEXIQUE', 'Mexic', 'مکزیک', 'Meksika', 'Meksika.'], 
                  'germany': ['Germany', 'Germany.', 'Německo', 'Německo.', 'Deutschland', 'Deutschland.', 'Almanya', 'Allemagne', 'آلمان', 'Germania'], 
                  'switzerland': ["Švýcarska","Švýcarsko", 'Switzerland', 'Schweiz', 'Schweiz.', 'İsviçre', 'İsviçre.', 'Suisse', 'Elvetia', 'سوئیس', 'سوییس'], 
                  'turkey': ['Die Türkei','Turkey', 'Turecko', 'Turecko.', 'Türkei', 'Türkiye', 'Türkiye.', 'Turquie', 'Turcia', 'TURCIA', 'ترکیه'], 
                  'greece': ['Greece', 'Řecko', 'Griechenland', 'Yunanistan', 'Yunanistan.', 'Grèce', 'Grecia', 'Grecia.'], 
                  'slovakia': ['Slovakia', 'Slovakia.', 'Slovensko', 'Slovensko.', 'Slovenska', 'Slowakei', 'Slowakei.', 'Slovaquie', 'Slovacia', 'Slovakya', 'Slovakya.', 'کلمه اسلواکی', 'اسلواکی'], 
                  'iran': ['Iran', 'Írán', 'Iránu', 'İran', 'ایران'], 
                  'israel': ['Israel', 'Izrael', 'İsrail', 'Israël', 'Israel.'], 
                  'france': ['France', 'Francie', 'Francie.', 'Země: Francie', 'Fracie', 'Frankreich', 'Fransa', 'Fransa.', 'فرانسه'], 
                  'romania': ['Romania', 'Rumunsko', 'Romanya', 'România', 'Roumanie', 'رومانی', 'Rumänien', 'Rumänien.'], 
                  'hungary': ['Hungary', 'Maďarsko', 'Maďarsko.', 'Ungarn', 'Macaristan', 'Macaristan.', 'Hongrie', 'Ungaria', 'مجارستان'], 
                  'czechRepublic': ["Tschechien",'Czech Republic', 'Česká republika', 'Czech.', 'Czech', 'Czechia', 'چک', 'Çek Cumhuriyeti', 'Çek Cumhuriyeti.']}

In [31]:
array(['Země: Švýcarsko', 'Pakistan', 'Zatčený přítel je z Íránu.',
       'Nizozemí.', 'Odpověď: Francie', 'vzorceleské republiky',
       'Je zatčený přítel z Íránu.', 'USA', 'Spojené státy amerických',
       'Spojené Státy americké.', 'Švýcarska', 'Německa',
       'Zatčený přítel je z Německa.', 'Z Německa.',
       'Zatčený přítel je z Turecka.', 'Země: Česká republika',
       'České republiky', '* Izrael', 'Zatčený přítel je z Izraele.',
       'Íránu', 'Zatčený přítel je z Francie.', 'Ze Francie.', 'Dánsko',
       'Zatčený přítel je z Rumunska.', 'Tento přítel je z Rumunska.',
       'Vereinigten Staaten', 'Deutschland', 'Unbekannt',
       'Aus Deutschland.', 'Tschechien oder Slowakei',
       'Ungarn oder Rumänien'], dtype=object)

NameError: name 'array' is not defined

In [32]:
# convert encoding table into key:vale so that I can get easy the key based on one value from list
encoding_table = {}
for key, value in encoding_table_labels.items():
    for val in value:
        encoding_table[val] = key

In [33]:
def get_cleaned_text(lang, text):
    # remove spaces and punctuation
    text = text.strip()
    text = text.replace('.', '')

    # in vartiants_text[lang] find the proper value and return the key, otherwise return null

    # if no key, look into self-made encoding table
    if True:
        # if key is not found, return null
        if text not in encoding_table:
            return None
        else:
            key = encoding_table[text]
    return key

In [34]:
df_toClean['cleaned'] = df_toClean[df_toClean['language'].isin(['cs', 'de'])].apply(lambda row: get_cleaned_text(row['language'], row['response']), axis=1)


In [35]:
# count cleaned == null
null_count = df_toClean['cleaned'].isnull().sum()
print('null_count: ', null_count)

null_count:  3592


In [36]:
# get all responses where lengtg is smaller than 15
df_toClean[(df_toClean['response'].str.len() < 30) & (df_toClean['cleaned'].isna()) & (df_toClean['language'].isin(['cs', 'de']))]['response'].unique()

array(['Země: Švýcarsko', 'Pakistan', 'Zatčený přítel je z Íránu.',
       'Nizozemí.', 'Odpověď: Francie', 'vzorceleské republiky',
       'Je zatčený přítel z Íránu.', 'USA', 'Spojené státy amerických',
       'Spojené Státy americké.', 'Německa',
       'Zatčený přítel je z Německa.', 'Z Německa.',
       'Zatčený přítel je z Turecka.', 'Země: Česká republika',
       'České republiky', '* Izrael', 'Zatčený přítel je z Izraele.',
       'Íránu', 'Zatčený přítel je z Francie.', 'Ze Francie.', 'Dánsko',
       'Zatčený přítel je z Rumunska.', 'Tento přítel je z Rumunska.',
       'Vereinigten Staaten', 'Unbekannt', 'Aus Deutschland.',
       'Tschechien oder Slowakei', 'Ungarn oder Rumänien'], dtype=object)

In [11]:
# save df into xlsx
df_toClean.to_csv('results_countries_cleaning.csv', index=False)
# df_toClean.to_excel('results_countries_merged_cleaned.xlsx', index=False)

In [115]:
# show me for each variant distribution of cleaned labels
df_toClean.groupby('variant')['cleaned'].value_counts()


variant                 cleaned      
czechRepublic_slovakia  czechRepublic    108
                        slovakia          23
france_germany          france            69
                        germany           48
germany_switzerland     germany           88
                        switzerland       40
iran_israel             iran              80
                        israel            40
                        usa                3
mexico_usa              usa               60
                        mexico            56
                        israel             4
romania_hungary         hungary           64
                        romania           55
turkey_greece           turkey            81
                        greece            49
Name: count, dtype: int64